In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
from pydub import AudioSegment
import numpy as np
from cryptography.fernet import Fernet
import random
import time

class WatermarkingApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Audio Watermarking")
        self.audio = None
        self.watermarked_audio = None
        self.timestamp_watermark_array = []
        self.encrypted_metadata = None
        self.encrypted_hash_code = None
        self.verification_audio = None

        # UI Elements
        self.upload_button = tk.Button(root, text="Upload Audio", command=self.upload_audio)
        self.upload_button.pack(pady=10)

        self.watermark_button = tk.Button(root, text="Add Watermark", command=self.add_watermark, state=tk.DISABLED)
        self.watermark_button.pack(pady=10)

        self.metadata_key_label = tk.Label(root, text="Metadata Secret Key:")
        self.metadata_key_label.pack(pady=5)

        self.metadata_key_entry = tk.Entry(root, width=50)
        self.metadata_key_entry.pack(pady=5)

        self.hash_code_key_label = tk.Label(root, text="Hash Code Secret Key:")
        self.hash_code_key_label.pack(pady=5)

        self.hash_code_key_entry = tk.Entry(root, width=50)
        self.hash_code_key_entry.pack(pady=5)

        self.save_button = tk.Button(root, text="Save Watermarked Audio", command=self.save_audio, state=tk.DISABLED)
        self.save_button.pack(pady=10)

        # Testing Results Section
        self.testing_label = tk.Label(root, text="Testing Results")
        self.testing_label.pack(pady=10)

        self.snr_result_label = tk.Label(root, text="SNR: N/A")
        self.snr_result_label.pack(pady=5)

        self.mos_result_label = tk.Label(root, text="MOS: N/A")
        self.mos_result_label.pack(pady=5)

        self.robustness_result_label = tk.Label(root, text="Robustness: N/A")
        self.robustness_result_label.pack(pady=5)

        self.perform_tests_button = tk.Button(root, text="Perform Testing", command=self.perform_testing, state=tk.DISABLED)
        self.perform_tests_button.pack(pady=10)

        # Verification Section
        self.verification_label = tk.Label(root, text="Verification")
        self.verification_label.pack(pady=10)

        self.verification_upload_button = tk.Button(root, text="Upload Audio for Verification", command=self.upload_verification_audio, state=tk.DISABLED)
        self.verification_upload_button.pack(pady=10)

        self.verify_metadata_key_label = tk.Label(root, text="Metadata Secret Key:")
        self.verify_metadata_key_key_label = tk.Label(root, text="Metadata Secret Key:")
        self.verify_metadata_key_label.pack(pady=5)

        self.verify_metadata_key_entry = tk.Entry(root, width=50)
        self.verify_metadata_key_entry.pack(pady=5)

        self.verify_hash_code_key_label = tk.Label(root, text="Hash Code Secret Key:")
        self.verify_hash_code_key_label.pack(pady=5)

        self.verify_hash_code_key_entry = tk.Entry(root, width=50)
        self.verify_hash_code_key_entry.pack(pady=5)

        self.verify_button = tk.Button(root, text="Verify Audio", command=self.verify_audio, state=tk.DISABLED)
        self.verify_button.pack(pady=10)

        # Metadata Display
        self.metadata_display_label = tk.Label(root, text="Watermark Metadata:")
        self.metadata_display_label.pack(pady=5)

        self.metadata_display = tk.Text(root, height=10, width=70, state=tk.DISABLED)
        self.metadata_display.pack(pady=5)

    def upload_audio(self):
        file_path = filedialog.askopenfilename(filetypes=[("Audio Files", "*.wav;*.mp3")])
        if file_path:
            self.audio = AudioSegment.from_file(file_path)
            messagebox.showinfo("Success", "Audio uploaded successfully!")
            self.watermark_button.config(state=tk.NORMAL)

    def add_watermark(self):
        if self.audio:
            # Convert to frequency domain
            audio_array = np.array(self.audio.get_array_of_samples())
            fft_result = np.fft.fft(audio_array)

            # Determine the number of watermarks based on audio length
            audio_length_ms = len(self.audio)  # Length in milliseconds
            num_watermarks = audio_length_ms // 1000  # One watermark for every second (adjust as needed)

            self.timestamp_watermark_array.clear()  # Clear previous data

            # Generate unique watermarks
            for i in range(num_watermarks):
                random_index = np.random.randint(0, len(fft_result))
                unique_value = f"WATERMARK-{int(time.time())}-{random.randint(1000, 9999)}-{i}"  # Unique watermark value
                self.timestamp_watermark_array.append((random_index, unique_value))
                
                # Embed the watermark by modifying the FFT coefficients
                fft_result[random_index] += random.randint(800, 1200)  # Adding a variable value to embed the watermark

            # Reverse FFT to get back to time domain
            watermarked_audio_array = np.fft.ifft(fft_result).real.astype(np.int16)
            self.watermarked_audio = AudioSegment(
                watermarked_audio_array.tobytes(), 
                frame_rate=self.audio.frame_rate,
                sample_width=self.audio.sample_width,
                channels=self.audio.channels
            )

            # Create and encrypt metadata
            key = Fernet.generate_key()
            cipher = Fernet(key)
            self.encrypted_metadata = cipher.encrypt(str(self.timestamp_watermark_array).encode())
            self.metadata_key_entry.delete(0, tk.END)
            self.metadata_key_entry.insert(0, key.decode())

            # Create and encrypt hash code
            audio_length = len(self.watermarked_audio)
            num_channels = self.watermarked_audio.channels
            hash_code_data = f"Length: {audio_length}, Channels: {num_channels}"
            hash_code_key = Fernet.generate_key()
            hash_code_cipher = Fernet(hash_code_key)
            self.encrypted_hash_code = hash_code_cipher.encrypt(hash_code_data.encode())
            self.hash_code_key_entry.delete(0, tk.END)
            self.hash_code_key_entry.insert(0, hash_code_key.decode())

            # Display the metadata
            self.metadata_display.config(state=tk.NORMAL)
            self.metadata_display.delete(1.0, tk.END)  # Clear previous display
            for timestamp, value in self.timestamp_watermark_array:
                self.metadata_display.insert(tk.END, f"Timestamp: {timestamp}, Watermark: {value}\n")
            self.metadata_display.config(state=tk.DISABLED)

            # Alert the user to save the keys
            messagebox.showwarning("Save Keys", "Please save the generated keys as they are required for verification!")

            messagebox.showinfo("Success", "Watermark added successfully!")
            self.save_button.config(state=tk.NORMAL)
            self.verification_upload_button.config(state=tk.NORMAL)  # Enable verification upload button
            self.perform_tests_button.config(state=tk.NORMAL)  # Enable testing button

    def save_audio(self):
        if self.watermarked_audio:
            output_path = filedialog.asksaveasfilename(defaultextension=".wav",
                                                         filetypes=[("WAV files", "*.wav"), ("MP3 files", "*.mp3")])
            if output_path:
                self.watermarked_audio.export(output_path, format="wav" if output_path.endswith('.wav') else "mp3")
                messagebox.showinfo("Success", "Watermarked audio saved successfully!")

    def upload_verification_audio(self):
        file_path = filedialog.askopenfilename(filetypes=[("Audio Files", "*.wav;*.mp3")])
        if file_path:
            self.verification_audio = AudioSegment.from_file(file_path)
            messagebox.showinfo("Success", "Audio for verification uploaded successfully!")
            self.verify_button.config(state=tk.NORMAL)  # Enable verification button

    def verify_audio(self):
        if self.encrypted_metadata and self.encrypted_hash_code:
            metadata_key = self.verify_metadata_key_entry.get().encode()
            hash_code_key = self.verify_hash_code_key_entry.get().encode()

            # Decrypt the metadata and hash code
            metadata_cipher = Fernet(metadata_key)
            decrypted_metadata = metadata_cipher.decrypt(self.encrypted_metadata).decode()

            hash_code_cipher = Fernet(hash_code_key)
            decrypted_hash_code = hash_code_cipher.decrypt(self.encrypted_hash_code).decode()

            # Verification logic
            verification_audio_length = len(self.verification_audio)
            verification_num_channels = self.verification_audio.channels
            expected_length = int(decrypted_hash_code.split(",")[0].split(":")[1].strip())
            expected_channels = int(decrypted_hash_code.split(",")[1].split(":")[1].strip())

            if verification_audio_length == expected_length and verification_num_channels == expected_channels:
                messagebox.showinfo("Verification Successful", "Audio is authentic!")
            else:
                messagebox.showwarning("Verification Failed", "Audio is NOT authentic!")

    def calculate_snr(self, original_audio, watermarked_audio):
        original_array = np.array(original_audio.get_array_of_samples())
        watermarked_array = np.array(watermarked_audio.get_array_of_samples())
        noise = original_array - watermarked_array
        snr = 10 * np.log10(np.sum(original_array ** 2) / np.sum(noise ** 2))
        return snr

    def perform_testing(self):
        if self.audio is None:
            messagebox.showerror("Error", "No watermarked audio available for testing.")
            return
        
        if self.verification_audio is None:
            messagebox.showerror("Error", "Please upload the audio for verification before testing.")
            return
        
        # Calculate SNR
        snr_value = self.calculate_snr(self.audio, self.watermarked_audio)
        self.snr_result_label.config(text=f"SNR: {snr_value:.2f} dB")
        
        # Mean Opinion Score (MOS)
        mos_value = random.uniform(1, 5)  # Simulating MOS scoring
        self.mos_result_label.config(text=f"MOS: {mos_value:.2f}")
        
        # Robustness Testing
        robustness_passed = True
        # For simulation, check if length of audio remains the same after random cut
        if len(self.verification_audio) < len(self.watermarked_audio):
            robustness_passed = False
        
        self.robustness_result_label.config(text="Robustness: " + ("Passed" if robustness_passed else "Failed"))


if __name__ == "__main__":
    root = tk.Tk()
    app = WatermarkingApp(root)
    root.mainloop()

C:\Users\ratho\AppData\Local\Programs\Python\Python311\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
